In [1]:
import random
import syft as sy
from syft.core import utils
from syft.core.frameworks.torch import utils as torch_utils
from syft.core.frameworks import encode
from syft.core.frameworks.torch.tensor import _MPCTensor
from syft.core.frameworks.torch.tensor import _GeneralizedPointerTensor
from syft.mpc import spdz
import torch
import torch.nn.functional as F
from torch.autograd import Variable as Var
import json

hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)
james = sy.VirtualWorker(id="james", hook=hook, is_client_worker=False)

bob.add_workers([alice, james])
alice.add_workers([bob, james])
james.add_workers([bob, alice])

In [2]:
x = sy.Variable(torch.FloatTensor([5, 6]), requires_grad=True)
y = sy.Variable(torch.FloatTensor([3, 4]), requires_grad=True)
x = sy._PlusIsMinusTensor().on(x)
y = sy._PlusIsMinusTensor().on(y)
z = x.add(y).sum()
z.backward()

# cut chain for the equality check
x.grad.data.child = x.grad.data.child.child
assert torch.equal(x.grad.data, torch.FloatTensor([1, 1]))